# Implementação MLP para dataset

In [59]:
# Função de ativação Logit

def f_net(activation):
     return 1.0 / (1.0 + np.exp(-activation))

In [60]:
# Derivada da Logit

def f_prime_net(f_net):
    return f_net*(1-f_net)

In [61]:
# Etapa Foward

def f_Forward(Xp,w_hidden_layer1,w_hidden_layer2,w_output_layer,net_hidden1,fnet_hidden1,net_hidden2,fnet_hidden2,net_out, fnet_out):
    # Hidden Layer 1
    net_hidden1  = np.sum(w_hidden_layer1*Xp,axis=1)
    fnet_hidden1 = f_net(net_hidden1)
    f_prime_hp1  = f_prime_net(fnet_hidden1)
    
    # Hidden Layer 2
    net_hidden2  = np.sum(np.r_[fnet_hidden1, 1]*w_hidden_layer2,axis=1)
    fnet_hidden2 = f_net(net_hidden2)
    f_prime_hp2 = f_prime_net(fnet_hidden2)
    
    # Output Layer
    net_out = np.sum(np.r_[fnet_hidden2, 1]*w_output_layer,axis=1)
    fnet_out =  f_net(net_out)
    f_prime_op = f_prime_net(fnet_out)
    
    return net_hidden1, fnet_hidden1, net_hidden2, fnet_hidden2, net_out, fnet_out, f_prime_op, f_prime_hp1, f_prime_hp2



In [62]:
# Backpropagation

def mlp_backpropagation(X,Y,n_inputs,n_hidden1,n_hidden2,n_outputs ,eta ,Erro_Global):
    # Vetor de entrada com acréscimo de coluna para bias
    Xp = np.concatenate((X, np.reshape(np.ones(len(X)),(len(X),1))), axis=1)

    # Inicialização dos pesos de forma aleatória
    w_hidden_layer1 = np.array([[random.uniform(-0.5,0.5) for i in range(n_inputs+1)] for i in range(n_hidden1)])
    w_hidden_layer2 = np.array([[random.uniform(-0.5,0.5) for i in range(n_hidden1+1)] for i in range(n_hidden2)])
    w_output_layer = np.array([[random.uniform(-0.5,0.5) for i in range(n_hidden2+1)]     for i in range(n_outputs)])

    ## Hidden Layer 1
    net_hidden1  = np.zeros(w_hidden_layer1.shape)
    fnet_hidden1 = np.zeros(w_hidden_layer1.shape)

    ## Hidden Layer 2
    net_hidden2  = np.zeros(w_hidden_layer2.shape)
    fnet_hidden2 = np.zeros(w_hidden_layer2.shape)
    
    ## Output Layer
    net_out  = np.zeros(n_outputs)
    fnet_out = np.zeros(n_outputs)
    # Vetor de saída da hidden com acréscimo de coluna para bias

    counter = 0
    squaredError = Erro_Global * 2

    while squaredError > Erro_Global :
        squaredError = 0
        for i in range(len(X)):
            net_hidden1, fnet_hidden1, net_hidden2, fnet_hidden2, net_out, fnet_out, f_prime_op, f_prime_hp1, f_prime_hp2 = f_Forward(Xp[i],w_hidden_layer1,w_hidden_layer2,w_output_layer,net_hidden1,fnet_hidden1,net_hidden2,fnet_hidden2,net_out, fnet_out)  
        
        
            error = Y[i] - fnet_out
            squaredError = squaredError + np.sum(error**2)

            #Cálculo dos deltas
            # Output
            delta_op  = error * f_prime_op   
            # Hidden 2
            w_o_kj_2   = w_output_layer[:,0:w_output_layer.shape[1]-1]
            delta_h_p2 = f_prime_hp2*np.dot(w_o_kj_2.T,delta_op)
            # Hidden 1
            w_o_kj_1   = w_hidden_layer2[:,0:w_hidden_layer2.shape[1]-1]
            delta_h_p1 = f_prime_hp1*np.dot(w_o_kj_1.T,delta_h_p2)
            
            #Treinamento
            w_output_layer  = w_output_layer  + eta * np.dot(np.reshape(delta_op,(len(w_output_layer),1)), np.reshape(np.r_[fnet_hidden2,1],(1,n_hidden2+1))) 
            w_hidden_layer2 = w_hidden_layer2 + eta * np.dot(np.reshape(delta_h_p2,(n_hidden2,1)),np.reshape(np.r_[fnet_hidden1,1],(1,n_hidden1+1))) 
            w_hidden_layer1 = w_hidden_layer1 + eta * np.dot(np.reshape(delta_h_p1,(n_hidden1,1)),np.reshape(Xp[i],(1,len(X[0])+1)))
            
        squaredError = squaredError / len(X)
        print(squaredError)
        counter = counter + 1
    
    return  squaredError , w_output_layer, w_hidden_layer1, w_hidden_layer2, fnet_out, counter

In [64]:
# Predição de novos dados

def f_predict(Xp,w_hidden_layer1,w_hidden_layer2,w_output_layer):
   
    return np.round(f_net(np.sum(np.r_[f_net(np.sum(np.r_[f_net(np.sum(w_hidden_layer1*Xp,axis=1)), 1]*w_hidden_layer2,axis=1)), 1]*w_output_layer,axis=1)))


In [63]:
import numpy as np
from math import exp
import random
import pandas as pd
#tratamentos de dataframes
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

random.seed(123)

# Importação do conjunto de dados Wine sem cabeçalho
df = pd.read_csv("/home/jovyan/MLPxRBF/wine.csv",delimiter=',',header=None)

# Nomeação das colunas
dict_colnames = {
0: 'type',
1: 'alcohol',
2: 'malic_acid',
3: 'ash',
4: 'alcalinity_ash',
5: 'magnesium',
6: 'total_phenols',
7: 'flavanoids',
8: 'nonflavanoid_phenols',
9: 'proanthocyanins',
10: 'color_intensity',
11: 'hue',
12: 'OD280_OD315_diluted_wines',
13: 'proline'
   }
    
df = df.rename(columns= dict_colnames)

# Criação do vetor de base canônica
df['p1'] = np.where(df['type']==1, 1, 0)
df['p2'] = np.where(df['type']==2, 1, 0)
df['p3'] = np.where(df['type']==3, 1, 0)

# Separação entre dados de treino e teste
lista = ['type','p1','p2','p3']
lista2 = ['p1','p2','p3']

x_train, x_test, y_train, y_test = train_test_split(df.drop(lista, axis = 1), 
                                                    df[lista2], 
                                                    test_size = 0.3, 
                                                    random_state = random.seed(123))

# Normalização dos dados
x_scaled = preprocessing.scale(x_train)
scaler = preprocessing.StandardScaler().fit(x_train)
x_train_norm = scaler.transform(x_train)
x_test_norm = scaler.transform(x_test)


# Matriz para treinar a rede
X = x_train_norm.copy()
Y = np.array(y_train.copy())

## Dimensões da Rede
n_inputs = len(X[0])
n_outputs = len(Y[0])
n_hidden1 = 3
n_hidden2 = 3


Erro, w_output_layer, w_hidden_layer1, w_hidden_layer2, fnet_out, counter = mlp_backpropagation(X,Y,n_inputs,n_hidden1,n_hidden2,n_outputs,eta = 0.01,Erro_Global=0.001)


0.6773963940358164
0.6736400371593152
0.6707551825614593
0.6685218892003781
0.666779231293276
0.6654088470777696
0.6643230867196219
0.6634565520539834
0.6627600672060217
0.662196369475851
0.6617370130104441
0.661360128673691
0.6610487910567036
0.660789818700228
0.6605728856581904
0.6603898585394916
0.6602342981277859
0.6601010820649248
0.6599861172668318
0.659886119337436
0.6597984423577167
0.659720946804539
0.659651896513681
0.6595898778991598
0.6595337363241008
0.659482525760064
0.6594354687939014
0.6593919247306266
0.659351364059274
0.6593133479410915
0.6592775116778762
0.6592435513466373
0.6592112129622983
0.6591802836657744
0.6591505845400087
0.6591219647385771
0.6590942966757108
0.6590674720770081
0.659041398729951
0.6590159978048195
0.6589912016416558
0.6589669519188756
0.6589431981350797
0.6589198963484221
0.6588970081281779
0.6588744996814745
0.6588523411248437
0.658830505875724
0.6588089701434576
0.658787712502928
0.6587667135369534
0.6587459555359437
0.6587254222453081
0.658

0.29986545553406363
0.2982994676585397
0.29673558817009854
0.29517341571835276
0.29361256085719445
0.29205264661393104
0.2904933090924977
0.28893419810767107
0.28737497784629507
0.28581532755047323
0.28425494221652486
0.28269353330227726
0.2811308294339477
0.279566577102583
0.27800054133873353
0.27643250635288236
0.2748622761281389
0.2732896749509576
0.2717145478652085
0.2701367610348918
0.26855620200122915
0.266972779820822
0.2653864250730807
0.2637970897272242
0.26220474686176864
0.26060939023256996
0.2590110336889963
0.2574097104416413
0.25580547218892735
0.2541983881138618
0.2525885437659051
0.2509760398462059
0.24936099091718678
0.24774352405949512
0.24612377750055656
0.24450189923932392
0.2428780456913068
0.2412523803766142
0.23962507267165942
0.23799629664245991
0.23636622997428833
0.2347350530089477
0.23310294789734218
0.23147009787144177
0.22983668663635767
0.22820289788017117
0.2265689148964858
0.22493492031246098
0.2233010959133669
0.22166762255348313
0.22003468014240535
0.2

0.02086995448565619
0.020797613213673707
0.02072572017012382
0.02065427149576756
0.02058326337281467
0.02051269202440549
0.020442553714100045
0.020372844745374253
0.020303561461122686
0.020234700243168202
0.020166257511778255
0.02009822972518764
0.020030613379127885
0.01996340500636288
0.019896601176231
0.019830198494193117
0.019764193601387276
0.019698583174188863
0.019633363923777125
0.01956853259570741
0.0195040859694893
0.019440020858170377
0.019376334107925833
0.0193130225976533
0.019250083238573773
0.019187512973837185
0.019125308778133936
0.01906346765731153
0.019001986647996186
0.018940862817219923
0.018880093262052422
0.018819675109238078
0.01875960551483768
0.018699881663875373
0.018640500769989778
0.018581460075090542
0.018522756849018978
0.0184643883892134
0.018406352020379133
0.018348645094162817
0.018291264988831237
0.01823420910895419
0.018177474885091915
0.01812105977348651
0.018064961255757687
0.0180091768386024
0.0179537040534986
0.01789854045641284
0.0178436836275120

0.008340986888742656
0.00832767139378879
0.00831439598724307
0.008301160494085698
0.008287964740290548
0.008274808552818232
0.008261691759609205
0.00824861418957709
0.008235575672601858
0.00822257603952316
0.008209615122133671
0.008196692753172545
0.008183808766318947
0.008170962996185456
0.008158155278311754
0.008145385449158228
0.00813265334609965
0.00811995880741893
0.008107301672300896
0.008094681780826108
0.00808209897396482
0.00806955309357083
0.008057043982375545
0.008044571483981945
0.008032135442858728
0.008019735704334383
0.00800737211459138
0.007995044520660433
0.007982752770414745
0.007970496712564296
0.007958276196650254
0.007946091073039337
0.007933941192918298
0.00792182640828838
0.007909746571959867
0.007897701537546684
0.007885691159460972
0.007873715292907852
0.007861773793880002
0.007849866519152508
0.007837993326277578
0.007826154073579462
0.007814348620149226
0.007802576825839757
0.007790838551260644
0.007779133657773201
0.007767462007485476
0.007755823463247334
0.

0.005073595777987861
0.005068422081679351
0.00506325847903618
0.005058104941213414
0.005052961439474192
0.005047827945189295
0.0050427044298366214
0.005037590865000674
0.005032487222372105
0.0050273934737471845
0.0050223095910273815
0.005017235546218783
0.005012171311431694
0.005007116858880148
0.0050020721608813766
0.00499703718985539
0.0049920119183245
0.0049869963189128255
0.004981990364345875
0.00497699402745001
0.00497200728115204
0.004967030098478769
0.004962062452556495
0.004957104316610598
0.004952155663965062
0.004947216468042046
0.00494228670236143
0.004937366340540392
0.004932455356292922
0.004927553723429404
0.004922661415856215
0.004917778407575226
0.0049129046726834324
0.004908040185372482
0.004903184919928266
0.004898338850730492
0.00489350195225225
0.004888674199059621
0.004883855565811229
0.004879046027257852
0.004874245558241964
0.004869454133697376
0.004864671728648796
0.004859898318211442
0.004855133877590608
0.004850378382081295
0.0048456318070677655
0.004840894128

...

0.0016605843833666022
0.001659987192621808
0.0016593904202194172
0.0016587940657254968
0.0016581981287067105
0.0016576026087303144
0.0016570075053641549
0.001656412818176677
0.0016558185467369078
0.001655224690614468
0.001654631249379575
0.0016540382226030255
0.001653445609856212
0.0016528534107111005
0.0016522616247402519
0.001651670251516809
0.0016510792906144975
0.001650488741607622
0.0016498986040710739
0.0016493088775803204
0.0016487195617114184
0.0016481306560409807
0.0016475421601462226
0.0016469540736049218
0.0016463663959954352
0.001645779126896688
0.0016451922658881869
0.0016446058125500025
0.0016440197664627878
0.001643434127207757
0.0016428488943666998
0.0016422640675219692
0.0016416796462564928
0.0016410956301537586
0.0016405120187978168
0.0016399288117733015
0.0016393460086653862
0.0016387636090598205
0.0016381816125429105
0.0016376000187015266
0.0016370188271231001
0.001636438037395617
0.0016358576491076263
0.0016352776618482295
0.0016346980752070931
0.001634118888774424

0.0014616868728719885
0.0014612211859813466
0.0014607557884914251
0.0014602906801357246
0.0014598258606480614
0.001459361329762579
0.0014588970872137469
0.001458433132736357
0.0014579694660655227
0.0014575060869366813
0.0014570429950855892
0.0014565801902483289
0.001456117672161296
0.0014556554405612096
0.0014551934951851165
0.001454731835770368
0.0014542704620546464
0.00145380937377595
0.0014533485706725905
0.0014528880524831993
0.00145242781894673
0.0014519678698024449
0.0014515082047899285
0.0014510488236490799
0.0014505897261201052
0.0014501309119435292
0.001449672380860201
0.001449214132611267
0.0014487561669382012
0.0014482984835827786
0.0014478410822870906
0.001447383962793551
0.0014469271248448681
0.0014464705681840704
0.0014460142925544898
0.0014455582976997789
0.0014451025833638884
0.0014446471492910867
0.0014441919952259445
0.0014437371209133432
0.0014432825260984794
0.0014428282105268385
0.00144237417394423
0.0014419204160967616
0.0014414669367308428
0.0014410137355932004
0

0.0013022804381275807
0.0013019087313205448
0.001301537231704412
0.0013011659391079422
0.0013007948533600773
0.0013004239742899469
0.001300053301726872
0.0012996828355003483
0.0012993125754400757
0.0012989425213759272
0.0012985726731379627
0.0012982030305564312
0.0012978335934617695
0.0012974643616845932
0.0012970953350557037
0.0012967265134060928
0.0012963578965669246
0.0012959894843695618
0.0012956212766455431
0.001295253273226584
0.0012948854739446012
0.001294517878631679
0.0012941504871200886
0.0012937832992422926
0.0012934163148309202
0.001293049533718795
0.0012926829557389154
0.0012923165807244678
0.0012919504085088104
0.0012915844389254922
0.0012912186718082432
0.0012908531069909675
0.0012904877443077494
0.0012901225835928547
0.0012897576246807397
0.0012893928674060206
0.001289028311603511
0.001288663957108188
0.0012882998037552257
0.0012879358513799588
0.001287572099817914
0.0012872085489047886
0.0012868451984764592
0.0012864820483689814
0.0012861190984185893
0.0012857563484616

0.0011754555815845437
0.0011751513202352445
0.0011748472128387097
0.0011745432592793713
0.0011742394594417773
0.0011739358132105927
0.0011736323204705938
0.001173328981106673
0.001173025795003841
0.001172722762047216
0.0011724198821220298
0.0011721171551136375
0.0011718145809074987
0.0011715121593891915
0.0011712098904444043
0.0011709077739589391
0.001170605809818714
0.0011703039979097528
0.0011700023381181987
0.0011697008303303146
0.0011693994744324625
0.0011690982703111267
0.0011687972178528914
0.0011684963169444696
0.0011681955674726712
0.0011678949693244278
0.0011675945223867837
0.001167294226546886
0.0011669940816919988
0.0011666940877094962
0.0011663942444868642
0.001166094551911704
0.001165795009871723
0.001165495618254735
0.0011651963769486776
0.0011648972858415857
0.0011645983448216079
0.0011642995537770105
0.0011640009125961643
0.0011637024211675468
0.0011634040793797532
0.0011631058871214846
0.00116280784428155
0.0011625099507488695
0.0011622122064124738
0.001161914611161499

0.0010686991479638825
0.001068446584833539
0.001068194138481137
0.001067941808826537
0.0010676895957896623
0.0010674374992905228
0.0010671855192491999
0.0010669336555858416
0.0010666819082206683
0.001066430277073978
0.001066178762066135
0.001065927363117578
0.0010656760801488227
0.001065424913080452
0.0010651738618331205
0.001064922926327556
0.0010646721064845623
0.001064421402225009
0.0010641708134698368
0.0010639203401400618
0.0010636699821567736
0.001063419739441127
0.0010631696119143563
0.0010629195994977538
0.0010626697021126961
0.0010624199196806248
0.0010621702521230578
0.0010619206993615767
0.0010616712613178358
0.001061421937913567
0.0010611727290705645
0.0010609236347106973
0.0010606746547559055
0.001060425789128193
0.0010601770377496422
0.001059928400542407
0.0010596798774287063
0.001059431468330829
0.001059183173171133
0.0010589349918720546
0.0010586869243560887
0.0010584389705458098
0.0010581911303638592
0.0010579434037329444
0.001057695790575846
0.0010574482908154105
0.00

## Predito

In [65]:
Xp = np.concatenate((x_test_norm, np.reshape(np.ones(len(x_test_norm)),(len(x_test_norm),1))), axis=1)

In [67]:
Y_pred =  np.zeros(y_test.shape)
for i in range(len(Xp[:,])):
    Y_pred[i] = f_predict(Xp[i],w_hidden_layer1,w_hidden_layer2,w_output_layer)

In [68]:
np.argmax(Y_pred,axis=1)

array([1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 1, 0, 1, 1, 0, 2, 1, 1, 0, 1,
       2, 0, 0, 2, 1, 2, 1, 0, 1, 0, 2, 1, 0, 2, 1, 1, 1, 0, 1, 1, 1, 2,
       0, 2, 2, 2, 0, 1, 0, 0, 1, 2])

## Real

In [69]:
np.argmax(np.array(y_test),axis=1)

array([1, 0, 1, 2, 1, 2, 1, 2, 1, 2, 0, 2, 1, 0, 1, 1, 0, 2, 1, 1, 1, 1,
       2, 0, 0, 2, 1, 2, 1, 0, 1, 0, 2, 1, 0, 2, 1, 1, 1, 0, 1, 1, 1, 2,
       0, 2, 2, 2, 0, 1, 0, 0, 1, 2])

# MLP

## Matriz de Confusão

In [70]:
from sklearn.metrics import confusion_matrix
confusion_matrix(np.argmax(np.array(y_test),axis=1), np.argmax(Y_pred,axis=1))

array([[14,  0,  0],
       [ 1, 23,  0],
       [ 0,  0, 16]])

## Acurácia

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(np.argmax(np.array(y_test),axis=1), np.argmax(Y_pred,axis=1))

0.9814814814814815